# Capstone: Pre-processing and Training Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
import sys

## Import of csv's created in nfl_capstone_EDA

In [3]:
year = pd.read_csv('teamstarterdraft.csv')
yearAV = pd.read_csv('teamstarterdraftAV.csv')
week = pd.read_csv('weekstarterdraft.csv')
weekAV = pd.read_csv('weekstarterdraftAV.csv')

### Creation of 2 additional dataframes that do not include the categorical variables of coach, offcoor, defcoor, off scheme, and def align.  If these do not add much to the variance, they are just preventing tree based algorithms from functioning well, as the number of dummy columns is large

In [5]:
yearnocoach = year.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])
yearnocoachAV = yearAV.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])

## Create pipeline for analysis

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

pipe = make_pipeline(StandardScaler(),
                    OneHotEncoder(),
                    Ridge(alpha=0.1))
